# Exploratory Data Analysis (EDA) for Personal Finance Data (v2)

**Objective:** Analyze the generated expense data (`dummy_expenses_generated.csv`) to validate its structure, adherence to generation rules, realism, and suitability for downstream AI/ML tasks (Phase 2 Assistant). Plots will be displayed inline.

**Data Source:** `../dummy_expenses_generated.csv` (Relative path from `reference/` to project root)
**Ruleset Reference:** `../sample_data_generation.csv`
**Metadata Reference:** `../expense_metadata.json`

In [16]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from pathlib import Path
import datetime
import logging
import os # Import os for directory creation
from typing import Optional, Dict, Any # For type hints

# Configure logging for analysis insights
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

In [17]:
# --- Setup Paths and Create Output Directory ---

# Define file paths relative to the notebook location (reference/)
NOTEBOOK_DIR = Path().resolve() # Should be the reference folder when running notebook there
PROJECT_ROOT = NOTEBOOK_DIR.parent
DATA_FILE = PROJECT_ROOT / "dummy_expenses_generated.csv"
RULES_FILE = PROJECT_ROOT / "sample_data_generation.csv"
METADATA_FILE = PROJECT_ROOT / "expense_metadata.json"

# Define image output directory (though we won't save by default now)
IMG_DIR = NOTEBOOK_DIR / "img"

# Optionally ensure the image directory exists if you uncomment saving later
# try:
#     os.makedirs(IMG_DIR, exist_ok=True)
#     logging.info(f"Image output directory exists or created: {IMG_DIR}")
# except OSError as e:
#     logging.error(f"Could not create image directory {IMG_DIR}: {e}")


## Helper Function: Load Rules

In [18]:
# --- Helper Function: Load Rules (Moved here to be defined before use) ---
def load_rules(filepath: Path) -> Optional[pd.DataFrame]:
    """Loads and preprocesses the ruleset CSV."""
    if not filepath.exists():
        logging.error(f"Rules file not found: {filepath}")
        return None
    try:
        df_rules = pd.read_csv(filepath)
        # Clean column names if needed (assuming they are already clean based on previous step)
        df_rules.columns = [col.strip() for col in df_rules.columns]
        # Split the valid types string into a list
        # Handle potential errors if the column doesn't exist or has NaNs
        if 'Valid-expense-types' in df_rules.columns:
             df_rules['Valid-expense-types'] = df_rules['Valid-expense-types'].fillna('').astype(str).str.split('|')
        else:
             logging.warning("'Valid-expense-types' column not found in rules file.")
             df_rules['Valid-expense-types'] = [[] for _ in range(len(df_rules))] # Add empty list

        # Convert amount/count columns to numeric, coercing errors
        for col in ['Min-expenses-amount', 'Max-expenses-amount', 'Max-times-per-month']:
            if col in df_rules.columns:
                df_rules[col] = pd.to_numeric(df_rules[col], errors='coerce')
            else:
                logging.warning(f"Column '{col}' not found in rules file.")
                df_rules[col] = np.nan # Add column with NaN if missing

        # Fill missing Max-times with a default (e.g., 5) or handle based on frequency
        df_rules['Max-times-per-month'].fillna(5, inplace=True)
        df_rules['Max-times-per-month'] = df_rules['Max-times-per-month'].astype(int)

        # Define critical columns needed from rules file
        critical_rule_cols = ['Category', 'Sub-category', 'User', 'Account', 'Expense-Frequency', 'Min-expenses-amount', 'Max-expenses-amount', 'Max-times-per-month']
        missing_critical = [col for col in critical_rule_cols if col not in df_rules.columns]
        if missing_critical:
            logging.error(f"Rules file is missing critical columns: {missing_critical}")
            return None

        # Drop rows with missing critical numeric info after conversion attempts
        df_rules.dropna(subset=['Min-expenses-amount', 'Max-expenses-amount'], inplace=True)
        # Drop rows with missing critical categorical info
        df_rules.dropna(subset=['Category', 'Sub-category', 'User', 'Account', 'Expense-Frequency'], inplace=True)

        logging.info(f"Loaded and preprocessed {len(df_rules)} rules from {filepath}")
        return df_rules
    except Exception as e:
        logging.error(f"Error loading or processing rules file {filepath}: {e}", exc_info=True)
        return None

## 1. Load Data and Basic Checks

Load the generated CSV, parse dates correctly using the 'dd-mm-yyyy' format, check basic info, data types, and null values.

In [19]:
logging.info(f"Attempting to load data from: {DATA_FILE}")
logging.info(f"Reference ruleset: {RULES_FILE}")
logging.info(f"Reference metadata: {METADATA_FILE}")

# Load the generated data
try:
    df = pd.read_csv(DATA_FILE)
    logging.info(f"Successfully loaded {DATA_FILE}. Shape: {df.shape}")
    print(f"Loaded {len(df)} rows.")
except FileNotFoundError:
    logging.error(f"Data file not found at {DATA_FILE}. Please ensure it's generated in the project root.")
    df = pd.DataFrame() # Create empty df to avoid downstream errors
except Exception as e:
    logging.error(f"Error loading data CSV {DATA_FILE}: {e}", exc_info=True)
    df = pd.DataFrame()

# Load metadata for validation checks
try:
    with open(METADATA_FILE, 'r') as f:
        metadata = json.load(f)
    logging.info(f"Successfully loaded metadata from {METADATA_FILE}")
except Exception as e:
    logging.error(f"Could not load metadata file {METADATA_FILE}: {e}")
    metadata = {} # Assign empty dict if load fails

INFO: Attempting to load data from: E:\Code\Projects\App-PersonalFinance\app-personal-finance\dummy_expenses_generated.csv
INFO: Reference ruleset: E:\Code\Projects\App-PersonalFinance\app-personal-finance\sample_data_generation.csv
INFO: Reference metadata: E:\Code\Projects\App-PersonalFinance\app-personal-finance\expense_metadata.json
INFO: Successfully loaded E:\Code\Projects\App-PersonalFinance\app-personal-finance\dummy_expenses_generated.csv. Shape: (951, 11)
INFO: Successfully loaded metadata from E:\Code\Projects\App-PersonalFinance\app-personal-finance\expense_metadata.json


Loaded 951 rows.


In [20]:
# Display basic information if data loaded
if not df.empty:
    print("\n--- Data Info ---")
    df.info()

    # Convert 'date' column from dd-mm-yyyy string to datetime objects for analysis
    print("\n--- Date Conversion ---")
    try:
        # Explicitly use dayfirst=True or format='%d-%m-%Y'
        df['date_dt'] = pd.to_datetime(df['date'], format='%d-%m-%Y', errors='coerce')
        logging.info("Converted 'date' column to datetime objects ('date_dt').")

        # Check for any conversion errors
        null_dates = df['date_dt'].isnull().sum()
        if null_dates > 0:
            logging.warning(f"Found {null_dates} rows with invalid date formats that couldn't be parsed to 'dd-mm-yyyy'.")
            # Optional: Display problematic rows
            # print("\nRows with date parsing errors:")
            # print(df[df['date_dt'].isnull()]['date'])
            df.dropna(subset=['date_dt'], inplace=True) # Remove rows with invalid dates for analysis
            logging.info(f"Removed rows with invalid dates. New shape: {df.shape}")
        else:
            logging.info("All dates successfully parsed.")

    except KeyError:
        logging.error("Column 'date' not found in the CSV.")
    except Exception as e:
        logging.error(f"Error during date conversion: {e}", exc_info=True)

else:
    print("DataFrame is empty. Cannot perform further analysis.")

INFO: Converted 'date' column to datetime objects ('date_dt').
INFO: All dates successfully parsed.



--- Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 951 entries, 0 to 950
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          951 non-null    object 
 1   year          951 non-null    int64  
 2   month         951 non-null    object 
 3   week          951 non-null    object 
 4   day_of_week   951 non-null    object 
 5   account       951 non-null    object 
 6   category      951 non-null    object 
 7   sub_category  951 non-null    object 
 8   type          951 non-null    object 
 9   user          951 non-null    object 
 10  amount        951 non-null    float64
dtypes: float64(1), int64(1), object(9)
memory usage: 81.9+ KB

--- Date Conversion ---


## 2. Data Range and Null Value Check

Verify the date range covers the expected period and check for unexpected missing values in core columns.

In [21]:
if not df.empty and 'date_dt' in df.columns:
    min_date = df['date_dt'].min()
    max_date = df['date_dt'].max()
    print(f"\n--- Date Range Check ---")
    print(f"Data spans from: {min_date.strftime('%Y-%m-%d')} to {max_date.strftime('%Y-%m-%d')}")

    expected_start = datetime.datetime(2023, 1, 1)
    expected_end = datetime.datetime(2025, 4, 20)

    if min_date.date() > expected_start.date():
        logging.warning(f"Data starts later ({min_date.date()}) than expected ({expected_start.date()}).")
    if max_date.date() < expected_end.date():
        logging.warning(f"Data ends earlier ({max_date.date()}) than expected ({expected_end.date()}).")

    print("\n--- Null Value Check ---")
    null_counts = df.isnull().sum()
    print("Columns with Null/NaN values:")
    print(null_counts[null_counts > 0]) # Only show columns with nulls

    # Check critical columns - adjust if 'sub_category' or 'type' can legitimately be empty strings instead of NaN
    critical_cols = ['date', 'year', 'month', 'week', 'day_of_week', 'account', 'category', 'user', 'amount', 'date_dt']
    unexpected_nulls = null_counts.reindex(critical_cols, fill_value=0).sum() > 0 # Use reindex to handle missing cols safely

    if unexpected_nulls:
        logging.error("UNEXPECTED null values found in critical columns! Review data generation.")
    else:
        logging.info("No unexpected null values found in critical columns (sub_category/type might have blanks/nulls).")
else:
     print("Skipping range/null checks as DataFrame is empty or date conversion failed.")

INFO: No unexpected null values found in critical columns (sub_category/type might have blanks/nulls).



--- Date Range Check ---
Data spans from: 2023-01-01 to 2025-04-15

--- Null Value Check ---
Columns with Null/NaN values:
Series([], dtype: int64)


## 3. Constraint Validation: Row Counts

Check if the number of transactions per month adheres to the `MONTHLY_MAX_ROWS = 100` limit.

In [22]:
if not df.empty and 'month' in df.columns:
    monthly_row_counts = df.groupby('month').size().reset_index(name='transaction_count')

    fig_row_counts = px.bar(monthly_row_counts, x='month', y='transaction_count',
                            title='Number of Transactions per Month',
                            labels={'month': 'Month (YYYY-MM)', 'transaction_count': 'Number of Transactions'},
                            height=400)
    fig_row_counts.add_hline(y=100, line_dash="dash", line_color="red", annotation_text="Max Limit (100)")
    fig_row_counts.update_layout(xaxis_tickangle=-45)
    fig_row_counts.show() # Display inline

    # Save image (Commented out)
    # try:
    #     img_path = IMG_DIR / "monthly_transaction_counts.png"
    #     fig_row_counts.write_image(img_path)
    #     logging.info(f"Saved monthly row counts plot to {img_path}")
    # except Exception as e:
    #     logging.error(f"Failed to save monthly row counts plot: {e}")


    # Discrepancy Check
    max_rows_in_month = monthly_row_counts['transaction_count'].max()
    print(f"\n--- Row Count Validation ---")
    print(f"Maximum rows found in a single month: {max_rows_in_month}")
    if max_rows_in_month > 100:
        exceeding_months = monthly_row_counts[monthly_row_counts['transaction_count'] > 100]['month'].tolist()
        logging.error(f"Monthly row count limit (>100) EXCEEDED! Max found: {max_rows_in_month}. Check months: {exceeding_months}")
    else:
        logging.info("Monthly row counts are within the limit (<= 100).")

    total_rows = df.shape[0]
    print(f"Total rows generated: {total_rows}")

else:
    print("Skipping row count validation.")

INFO: Monthly row counts are within the limit (<= 100).



--- Row Count Validation ---
Maximum rows found in a single month: 47
Total rows generated: 951


## 4. Constraint Validation: Monthly Totals

Verify if the total spending per month falls within the target range of ₹60,000 - ₹120,000.

In [23]:
if not df.empty and 'month' in df.columns and 'amount' in df.columns:
    monthly_totals = df.groupby('month')['amount'].sum().reset_index(name='total_amount')

    fig_monthly_totals = px.bar(monthly_totals, x='month', y='total_amount',
                                title='Total Spending per Month',
                                labels={'month': 'Month (YYYY-MM)', 'total_amount': 'Total Amount (INR)'},
                                height=400)
    fig_monthly_totals.add_hline(y=60000, line_dash="dash", line_color="orange", annotation_text="Min Target (60k)")
    fig_monthly_totals.add_hline(y=120000, line_dash="dash", line_color="red", annotation_text="Max Target (120k)")
    fig_monthly_totals.update_layout(yaxis_tickprefix="₹", xaxis_tickangle=-45)
    fig_monthly_totals.show() # Display inline

    # Save image (Commented out)
    # try:
    #     img_path = IMG_DIR / "monthly_total_spending.png"
    #     fig_monthly_totals.write_image(img_path)
    #     logging.info(f"Saved monthly total spending plot to {img_path}")
    # except Exception as e:
    #     logging.error(f"Failed to save monthly total spending plot: {e}")

    # Discrepancy Check
    min_monthly_total = monthly_totals['total_amount'].min()
    max_monthly_total = monthly_totals['total_amount'].max()
    months_below_min = monthly_totals[monthly_totals['total_amount'] < 60000]
    months_above_max = monthly_totals[monthly_totals['total_amount'] > 120000]

    print(f"\n--- Monthly Total Validation ---")
    print(f"Minimum monthly total found: ₹{min_monthly_total:,.2f}")
    print(f"Maximum monthly total found: ₹{max_monthly_total:,.2f}")

    if not months_below_min.empty:
        logging.warning(f"{len(months_below_min)} months found BELOW ₹60,000 target: {months_below_min['month'].tolist()}")
    else:
        logging.info("All monthly totals meet or exceed the minimum target (₹60,000).")

    if not months_above_max.empty:
        # Allow slightly above (e.g. 5%) due to fixed costs potentially pushing it over
        months_significantly_above_max = monthly_totals[monthly_totals['total_amount'] > 120000 * 1.05]
        if not months_significantly_above_max.empty:
            logging.error(f"{len(months_significantly_above_max)} months found SIGNIFICANTLY ABOVE ₹120,000 target (>5% overshoot): {months_significantly_above_max['month'].tolist()}")
        else:
            logging.warning(f"{len(months_above_max)} months found slightly above ₹120,000 target (<=5% overshoot - check if acceptable): {months_above_max['month'].tolist()}")
    else:
        logging.info("All monthly totals are within or acceptably close to the maximum target (<= ₹120,000).")
else:
    print("Skipping monthly total validation.")

INFO: All monthly totals meet or exceed the minimum target (₹60,000).
ERROR: 22 months found SIGNIFICANTLY ABOVE ₹120,000 target (>5% overshoot): ['2023-01', '2023-02', '2023-03', '2023-04', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-09', '2024-11', '2025-01', '2025-03']



--- Monthly Total Validation ---
Minimum monthly total found: ₹121,309.91
Maximum monthly total found: ₹183,334.33


## 5. Mapping Validation: User vs. Account

Ensure that the `user` column correctly corresponds to the `account` column based on the metadata rules.

In [24]:
if not df.empty and metadata and 'User' in metadata:
    print("\n--- User-Account Mapping Validation ---")
    # Display counts of each combination
    user_account_crosstab = pd.crosstab(df['user'], df['account'])
    print("Observed User-Account Combinations (Transaction Counts):")
    print(user_account_crosstab)

    # Discrepancy Check against metadata
    valid_map = metadata.get("User", {})
    invalid_combos_found = False
    checked_accounts = set()

    # Use tqdm if iterating over many rows, otherwise direct iteration is fine
    from tqdm.notebook import tqdm # Use notebook version
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Checking User-Account"):
    # for index, row in df.iterrows(): # Direct iteration for <2500 rows is okay
        account = row['account']
        user = row['user']
        # Avoid re-checking same account if metadata is missing
        if account in checked_accounts: continue

        expected_user = valid_map.get(account)

        if expected_user is None:
            logging.error(f"FATAL: Account '{account}' found in data but MISSING in metadata User map! Cannot validate.")
            invalid_combos_found = True
            checked_accounts.add(account) # Mark as checked to avoid repeating error
        elif user != expected_user:
            logging.error(f"User-Account MISMATCH! Account: '{account}', Expected User: '{expected_user}', Found User: '{user}'. Row index: {index}")
            invalid_combos_found = True
            # Don't add to checked_accounts here, allow finding more mismatches for same account if they exist

    if not invalid_combos_found:
        logging.info("User-Account mapping appears consistent with metadata for all transactions.")
    else:
        logging.error("CRITICAL User-Account mapping inconsistencies detected! Review generation logic or metadata.")
else:
    print("Skipping User-Account validation (Data or Metadata missing/incomplete).")


--- User-Account Mapping Validation ---
Observed User-Account Combinations (Transaction Counts):
account  Anirban-ICICI  Anirban-SBI  Puspita-Bandhan  Puspita-SBI
user                                                             
Anirban            412          101                0            0
Puspita              0            0               62          376


Checking User-Account:   0%|          | 0/951 [00:00<?, ?it/s]

INFO: User-Account mapping appears consistent with metadata for all transactions.


## 6. Mapping Validation: Category vs. Sub-Category

Check if all non-blank `sub_category` values are valid for their corresponding `category` based on the metadata.

In [25]:
if not df.empty and metadata and 'categories' in metadata:
    print("\n--- Category-SubCategory Mapping Validation ---")
    category_map = metadata.get("categories", {})
    invalid_subcats_found = False
    invalid_details = [] # Store details of invalid rows

    from tqdm.notebook import tqdm
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Checking Cat-SubCat"):
    # for index, row in df.iterrows():
        category = row['category']
        sub_category = row['sub_category']

        # Only validate if sub_category is present (not NaN or empty string)
        if pd.notna(sub_category) and sub_category != "":
            valid_subcats = category_map.get(category)

            if valid_subcats is None:
                detail = f"Category '{category}' (Row {index}) not found in metadata map."
                if detail not in [d[0] for d in invalid_details]: # Log category error once
                    logging.error(detail)
                    invalid_details.append((detail, index))
                invalid_subcats_found = True
            elif sub_category not in valid_subcats:
                detail = f"Invalid Sub-category! Category: '{category}', Found Sub-cat: '{sub_category}'. Valid: {valid_subcats}. Row Index: {index}"
                logging.error(detail)
                invalid_details.append((detail, index))
                invalid_subcats_found = True

    if not invalid_subcats_found:
        logging.info("Category-SubCategory mapping appears consistent with metadata for all non-blank sub-categories.")
    else:
        logging.error(f"CRITICAL Category-SubCategory mapping inconsistencies detected! Found {len(invalid_details)} issues.")
        # print("\nSample Invalid Details:")
        # for detail, idx in invalid_details[:min(5, len(invalid_details))]:
        #     print(f"- {detail}")
else:
    print("Skipping Category-SubCategory validation (Data or Metadata missing/incomplete).")


--- Category-SubCategory Mapping Validation ---


Checking Cat-SubCat:   0%|          | 0/951 [00:00<?, ?it/s]

INFO: Category-SubCategory mapping appears consistent with metadata for all non-blank sub-categories.


## 7. Fixed Expense Validation: Rent

Check frequency (1/month), amount (30k), payer (Anirban/ICICI).

In [26]:
if not df.empty and 'month' in df.columns:
    print("\n--- Rent Validation ---")
    rent_df = df[(df['category'] == 'Rent') & (df['sub_category'] == 'House Rent')].copy()

    if rent_df.empty:
        logging.warning("No Rent transactions found!")
    else:
        valid = True
        # Check Amount
        if not (rent_df['amount'] == 30000).all():
            logging.error(f"Rent amount incorrect! Found values != 30000: {rent_df[rent_df['amount'] != 30000]['amount'].unique()}")
            valid = False
        # Check User/Account
        if not ((rent_df['user'] == 'Anirban') & (rent_df['account'] == 'Anirban-ICICI')).all():
            logging.error(f"Rent payer incorrect! Found instances not paid by Anirban/Anirban-ICICI.")
            valid = False
        # Check Frequency
        rent_monthly_counts = rent_df.groupby('month').size()
        if (rent_monthly_counts != 1).any():
            logging.error(f"Rent frequency incorrect! Months with counts != 1: {rent_monthly_counts[rent_monthly_counts != 1].index.tolist()}")
            valid = False

        if valid:
            logging.info("Rent transactions appear correct (Amount, Payer, Frequency).")

        # Visualize counts even if correct
        fig_rent_counts = px.bar(rent_monthly_counts.reset_index(name='count'), x='month', y='count', title='Monthly Rent Transaction Count', height=300)
        fig_rent_counts.update_layout(yaxis_title='Count', xaxis_title='Month (YYYY-MM)', showlegend=False, yaxis=dict(tickmode='linear', dtick=1))
        fig_rent_counts.show() # Display inline
        # try: fig_rent_counts.write_image(IMG_DIR / "rent_monthly_counts.png") # Commented out
        # except Exception as e: logging.error(f"Failed to save rent counts plot: {e}")

else:
    print("Skipping Rent validation.")


--- Rent Validation ---


INFO: Rent transactions appear correct (Amount, Payer, Frequency).


## 8. Fixed Expense Validation: SIP & ULIP

Check monthly frequency, amounts (SIP=3k, ULIP=4k), and payer (Anirban-ICICI).

In [27]:
if not df.empty and 'month' in df.columns:
    print("\n--- SIP & ULIP Validation ---")
    sip_df = df[(df['category'] == 'Investment') & (df['sub_category'] == 'SIP')].copy()
    ulip_df = df[(df['category'] == 'Insurance Premium') & (df['sub_category'] == 'ULIP')].copy()
    valid_sip = True; valid_ulip = True

    # --- SIP ---
    if sip_df.empty: logging.warning("No SIP transactions found!")
    else:
        if not (sip_df['amount'] == 3000).all(): logging.error(f"SIP amount incorrect! Found values != 3000."); valid_sip = False
        if not ((sip_df['user'] == 'Anirban') & (sip_df['account'] == 'Anirban-ICICI')).all(): logging.error("SIP payer incorrect!"); valid_sip = False
        sip_counts = sip_df.groupby('month').size()
        if (sip_counts > 1).any(): logging.error(f"SIP frequency incorrect (>1 per month)!"); valid_sip = False
        # Check for missing months more accurately
        all_months_in_data = set(df['month'].unique()) # Get all unique months from the main dataframe
        sip_months_found = set(sip_df['month'].unique())
        missing_sip_months = all_months_in_data - sip_months_found
        if missing_sip_months and valid_sip: logging.warning(f"SIP seems missing in {len(missing_sip_months)} months: {sorted(list(missing_sip_months))[:5]}...")

        if valid_sip: logging.info("SIP transactions appear correct (Amount, Payer, Frequency<=1). Check warnings for missed months.")

    # --- ULIP ---
    if ulip_df.empty: logging.warning("No ULIP transactions found!")
    else:
        if not (ulip_df['amount'] == 4000).all(): logging.error(f"ULIP amount incorrect! Found values != 4000."); valid_ulip = False
        if not ((ulip_df['user'] == 'Anirban') & (ulip_df['account'] == 'Anirban-ICICI')).all(): logging.error("ULIP payer incorrect!"); valid_ulip = False
        ulip_counts = ulip_df.groupby('month').size()
        if (ulip_counts > 1).any(): logging.error(f"ULIP frequency incorrect (>1 per month)!"); valid_ulip = False
        ulip_months_found = set(ulip_df['month'].unique())
        missing_ulip_months = all_months_in_data - ulip_months_found
        if missing_ulip_months and valid_ulip: logging.warning(f"ULIP seems missing in {len(missing_ulip_months)} months: {sorted(list(missing_ulip_months))[:5]}...")

        if valid_ulip: logging.info("ULIP transactions appear correct (Amount, Payer, Frequency<=1). Check warnings for missed months.")

else:
    print("Skipping SIP/ULIP validation.")

INFO: SIP transactions appear correct (Amount, Payer, Frequency<=1). Check warnings for missed months.
INFO: ULIP transactions appear correct (Amount, Payer, Frequency<=1). Check warnings for missed months.



--- SIP & ULIP Validation ---


## 9. Fixed Expense Validation: Maid Salary

Check monthly frequency (once), amount (₹2500), and payer (Puspita-SBI).

In [28]:
if not df.empty and 'month' in df.columns:
    print("\n--- Maid Salary Validation ---")
    maid_df = df[(df['category'] == 'Household') & (df['sub_category'] == 'Maid')].copy()
    valid_maid = True

    if maid_df.empty: logging.warning("No Maid transactions found!")
    else:
        if not (maid_df['amount'] == 2500).all(): logging.error(f"Maid amount incorrect!"); valid_maid = False
        if not ((maid_df['user'] == 'Puspita') & (maid_df['account'] == 'Puspita-SBI')).all(): logging.error("Maid payer incorrect!"); valid_maid = False
        maid_counts = maid_df.groupby('month').size()
        if (maid_counts != 1).any(): logging.error(f"Maid frequency incorrect (!=1 per month)!"); valid_maid = False
        if valid_maid: logging.info("Maid Salary transactions appear correct.")
else:
    print("Skipping Maid Salary validation.")

INFO: Maid Salary transactions appear correct.



--- Maid Salary Validation ---


## 10. Ad-hoc Amount Distribution: Key Categories (Box Plot)

Visualize amount distributions for major variable spending areas.

In [29]:
if not df.empty and 'amount' in df.columns and 'category' in df.columns:
    print("\n--- Ad-hoc Amount Distributions (Box Plots) ---")
    major_variable_cats = ['Grocery', 'Restaurant', 'Shopping', 'Travel', 'Household', 'Health', 'Entertainment']
    plot_df = df[df['category'].isin(major_variable_cats)].copy() # Make a copy
    # Exclude known large fixed costs from Household/Travel for better visualization of variable costs
    plot_df = plot_df[~((plot_df['category'] == 'Household') & (plot_df['sub_category'] == 'Maid'))]
    # Exclude potential large vacation sums if they skew too much
    # plot_df = plot_df[~((plot_df['category'] == 'Travel') & (plot_df['sub_category'] == 'Vacation') & (plot_df['amount'] > 15000))] 
    # Exclude large furniture costs
    plot_df = plot_df[~((plot_df['category'] == 'Household') & (plot_df['sub_category'] == 'Furniture') & (plot_df['amount'] > 10000))]

    if not plot_df.empty:
        fig_box_cat = px.box(plot_df, x='category', y='amount', points="outliers",
                             title='Amount Distribution by Major Variable Category (Log Scale, Excl. Maid/Large Furniture/Vacation)',
                             labels={'category': 'Category', 'amount': 'Amount (INR)'},
                             log_y=True, height=500)
        fig_box_cat.update_layout(xaxis_tickangle=-45)
        fig_box_cat.show() # Display inline
        # try: fig_box_cat.write_image(IMG_DIR / "category_amount_boxplot.png") # Commented out
        # except Exception as e: logging.error(f"Failed to save category box plot: {e}")

        # Discrepancy Check:
        print(plot_df.groupby('category')['amount'].describe())
        logging.info("Review box plot/stats: Do medians/ranges seem realistic for Bangalore daily/weekly expenses in these categories?")
    else:
        logging.info("No data for major variable categories found after filtering fixed/large items.")
else:
    print("Skipping amount distribution analysis (missing columns).")


--- Ad-hoc Amount Distributions (Box Plots) ---


INFO: Review box plot/stats: Do medians/ranges seem realistic for Bangalore daily/weekly expenses in these categories?


               count         mean          std     min        25%       50%  \
category                                                                      
Entertainment   34.0  1858.921176  1037.274908  484.32  1039.2350  1554.955   
Grocery         83.0  1317.318313  1086.638861  118.41   398.4200   903.230   
Health          52.0  3298.279808  2711.537050  540.44  1153.2025  2264.335   
Household      120.0  2051.820250  2157.755727   70.99   781.8050  1444.155   
Restaurant      79.0   945.827595   831.638598   63.74   336.9350   573.160   
Shopping       122.0  4939.460984  2925.484306  307.33  2913.7725  4402.180   
Travel         153.0  4067.852810  5407.537065   53.94   439.1700  1878.090   

                     75%       max  
category                            
Entertainment  2444.9575   4763.75  
Grocery        2221.5200   4124.14  
Health         4809.3100   9084.03  
Household      2317.8000  11323.94  
Restaurant     1542.0300   3476.31  
Shopping       6655.8950  140

## 11. Ad-hoc Amount Distribution: Grocery Sub-Categories (Box Plot)

Compare spending ranges for different grocery platforms/stores.

In [30]:
if not df.empty and 'sub_category' in df.columns and 'category' in df.columns:
    print("\n--- Grocery Sub-Category Amount Distributions ---")
    grocery_subs = ['BigBasket', 'Zepto', 'Local Store', 'Amazon', 'Flipkart Grocery']
    plot_df_grocery = df[(df['category'] == 'Grocery') & (df['sub_category'].isin(grocery_subs))]

    if not plot_df_grocery.empty:
        fig_box_grocery = px.box(plot_df_grocery, x='sub_category', y='amount', points="outliers",
                                 title='Grocery Amount Distribution by Store/Platform (Log Scale)',
                                 labels={'sub_category': 'Store/Platform', 'amount': 'Amount (INR)'},
                                 category_orders={"sub_category": grocery_subs}, # Control order
                                 log_y=True, height=500)
        fig_box_grocery.show() # Display inline
        # try: fig_box_grocery.write_image(IMG_DIR / "grocery_subcat_boxplot.png") # Commented out
        # except Exception as e: logging.error(f"Failed to save grocery box plot: {e}")

        # Discrepancy Check
        print(plot_df_grocery.groupby('sub_category')['amount'].describe())
        logging.info("Check if distributions align with expectations (e.g., BigBasket/Amazon generally higher median/IQR than Zepto/Local Store).")
    else:
        logging.info("No data found for specified Grocery sub-categories.")
else:
    print("Skipping grocery sub-category analysis.")


--- Grocery Sub-Category Amount Distributions ---


INFO: Check if distributions align with expectations (e.g., BigBasket/Amazon generally higher median/IQR than Zepto/Local Store).


                  count         mean          std     min        25%      50%  \
sub_category                                                                    
Amazon             14.0  2358.546429  1011.778104  595.08  1767.9875  2322.53   
BigBasket          11.0  2400.901818   956.428750  742.93  1911.0250  2456.21   
Flipkart Grocery   11.0  2029.481818   902.174004  461.95  1360.9500  2332.46   
Local Store        12.0   742.185000   477.796766  144.54   383.4650   673.25   
Zepto              17.0   672.707059   429.387667  190.82   318.9000   599.55   

                        75%      max  
sub_category                          
Amazon            2888.7125  3972.13  
BigBasket         2833.8800  4124.14  
Flipkart Grocery  2730.6700  2921.10  
Local Store        990.9800  1749.74  
Zepto             1114.8800  1363.96  


## 12. Frequency Validation: Ad-hoc Monthly Counts

Check adherence to `Max-times-per-month` for specific ad-hoc rules (Waste, Gifts, Entertainment).

In [31]:
if not df.empty and 'month' in df.columns:
    print("\n--- Ad-hoc Monthly Frequency Validation (Waste/Gifts/Ent) ---")
    df_rules_loaded = load_rules(RULES_FILE) # Use the function defined earlier

    if df_rules_loaded is not None:
        # Define the rules we want to check explicitly based on the constraints given
        rules_to_check_criteria = (
            (df_rules_loaded['Expense-Frequency'] == 'ad-hoc') &
            (
                (df_rules_loaded['Category'] == 'Waste') |
                (df_rules_loaded['Category'] == 'Gifts & Donations') |
                (df_rules_loaded['Category'] == 'Entertainment')
            )
        )
        rules_to_check_df = df_rules_loaded[rules_to_check_criteria].copy()

        if not rules_to_check_df.empty:
            # Create a unique key combining relevant fields from BOTH data and rules
            # Use lowercase for robust matching if casing differs slightly
            df['rule_match_key'] = df.apply(lambda r: f"{r['category'].lower()}|{r['sub_category'].lower()}|{r['user'].lower()}|{r['account'].lower()}", axis=1)
            rules_to_check_df['rule_match_key'] = rules_to_check_df.apply(lambda r: f"{r['Category'].lower()}|{r['Sub-category'].lower()}|{r['User'].lower()}|{r['Account'].lower()}", axis=1)

            # Group transactions by month and rule_key to count actual occurrences
            monthly_actual_counts = df[df['rule_match_key'].isin(rules_to_check_df['rule_match_key'])].groupby(['month', 'rule_match_key']).size().reset_index(name='actual_count')

            # Merge actual counts with the limits from the rules file
            merged_counts = pd.merge(
                monthly_actual_counts,
                rules_to_check_df[['rule_match_key', 'Max-times-per-month']],
                on='rule_match_key',
                how='left' # Keep all actual counts, match rules where possible
            )
            # Drop rows where the rule couldn't be matched (shouldn't happen if keys are correct)
            merged_counts.dropna(subset=['Max-times-per-month'], inplace=True)
            merged_counts['Max-times-per-month'] = merged_counts['Max-times-per-month'].astype(int) # Ensure integer comparison

            # Find violations
            merged_counts['exceeded'] = merged_counts['actual_count'] > merged_counts['Max-times-per-month']
            exceeded_df = merged_counts[merged_counts['exceeded']]

            if not exceeded_df.empty:
                logging.error("Max-times-per-month constraint VIOLATED for some ad-hoc rules!")
                print("Violations Found (Rule Key | Month | Actual Count | Max Allowed):")
                # Improve display for readability
                exceeded_df_display = exceeded_df[['rule_match_key', 'month', 'actual_count', 'Max-times-per-month']].copy()
                exceeded_df_display['Rule_Info'] = exceeded_df_display['rule_match_key'].str.replace('|', ' | ', regex=False)
                print(exceeded_df_display[['Rule_Info', 'month', 'actual_count', 'Max-times-per-month']].to_string(index=False))
            else:
                logging.info("Ad-hoc frequency constraints (Max-times-per-month) for Waste/Gifts/Entertainment appear satisfied.")

            # Clean up added column from main df
            if 'rule_match_key' in df.columns: df.drop(columns=['rule_match_key'], inplace=True, errors='ignore')
        else:
            logging.info("No specific Waste/Gifts/Entertainment ad-hoc rules found in the rules file for frequency validation.")
    else:
        logging.warning("Could not load rules file, skipping ad-hoc frequency validation.")
else:
    print("Skipping ad-hoc frequency validation.")


--- Ad-hoc Monthly Frequency Validation (Waste/Gifts/Ent) ---


C:\Users\Teertha's PC\AppData\Local\Temp\ipykernel_11796\1882847337.py:28: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



INFO: Loaded and preprocessed 177 rules from E:\Code\Projects\App-PersonalFinance\app-personal-finance\sample_data_generation.csv
INFO: Ad-hoc frequency constraints (Max-times-per-month) for Waste/Gifts/Entertainment appear satisfied.


## 13. Account Usage Bias Check

Verify Puspita-Bandhan use for Travel/Beauty and Anirban-ICICI dominance.

In [32]:
if not df.empty and 'account' in df.columns and 'user' in df.columns and 'category' in df.columns:
    print("\n--- Account Usage Validation ---")

    # Overall Spend per Account (Pie Chart)
    account_totals = df.groupby('account')['amount'].sum().reset_index()
    fig_acc_pie = px.pie(account_totals, values='amount', names='account',
                         title='Overall Spending Proportion by Account', hole=0.3)
    fig_acc_pie.update_traces(textinfo='percent+label', hovertemplate="<b>%{label}</b><br>Total: ₹%{value:,.0f}<br>(%{percent})<extra></extra>")
    fig_acc_pie.show() # Display inline
    # try: fig_acc_pie.write_image(IMG_DIR / "account_spending_pie.png") # Commented out
    # except Exception as e: logging.error(f"Failed to save account pie plot: {e}")

    # Puspita-Bandhan: Travel & Beauty focus?
    puspita_bandhan_df = df[df['account'] == 'Puspita-Bandhan']
    if not puspita_bandhan_df.empty:
        pb_cat_dist = puspita_bandhan_df['category'].value_counts(normalize=True) * 100
        print("\nPuspita-Bandhan Category Distribution (% of transactions):")
        print(pb_cat_dist.round(1))
        allowed_cats = ['Travel', 'Beauty']
        other_cats_used = [cat for cat in pb_cat_dist.index if cat not in allowed_cats]
        if other_cats_used:
            logging.warning(f"Puspita-Bandhan account used for categories other than Travel/Beauty: {other_cats_used}")
        else:
             logging.info("Puspita-Bandhan usage appears focused on Travel/Beauty.")
        # Pie chart for Puspita-Bandhan
        fig_pb_cat = px.pie(puspita_bandhan_df, names='category', title='Puspita-Bandhan Spending by Category', hole=0.3)
        fig_pb_cat.update_traces(textinfo='percent+label')
        fig_pb_cat.show() # Display inline
        # try: fig_pb_cat.write_image(IMG_DIR / "puspita_bandhan_category_pie.png") # Commented out
        # except Exception as e: logging.error(f"Failed to save Puspita-Bandhan pie plot: {e}")
    else:
        logging.info("No transactions found for Puspita-Bandhan.")

    # Anirban-ICICI Dominance Check
    anirban_df = df[df['user'] == 'Anirban']
    if not anirban_df.empty:
         total_anirban_spend = anirban_df['amount'].sum()
         if total_anirban_spend > 0:
              icici_spend = anirban_df[anirban_df['account'] == 'Anirban-ICICI']['amount'].sum()
              icici_share = (icici_spend / total_anirban_spend) * 100
              print(f"\nAnirban-ICICI accounts for {icici_share:.1f}% of Anirban's total spending amount.")
              if icici_share < 60: # Arbitrary threshold
                   logging.warning("Anirban-ICICI share of Anirban's spending seems lower than expected (<60%).")
              else:
                   logging.info("Anirban-ICICI appears dominant for Anirban's spending (>=60%).")
         else:
              logging.info("Anirban has zero total spending.")
    else:
         logging.info("No transactions found for Anirban.")
else:
    print("Skipping account usage validation (missing columns).")


--- Account Usage Validation ---


INFO: Puspita-Bandhan usage appears focused on Travel/Beauty.



Puspita-Bandhan Category Distribution (% of transactions):
category
Travel    58.1
Beauty    41.9
Name: proportion, dtype: float64


INFO: Anirban-ICICI appears dominant for Anirban's spending (>=60%).



Anirban-ICICI accounts for 74.4% of Anirban's total spending amount.


## 14. Temporal Pattern: Spending Over Month Day

Check if Grocery/Shopping spending is concentrated early in the month.

In [33]:
if not df.empty and 'date_dt' in df.columns and 'category' in df.columns:
    print("\n--- Intra-Month Spending Pattern (Grocery/Shopping) ---")
    df['day_of_month'] = df['date_dt'].dt.day
    # Filter for relevant categories AND non-fixed amounts (exclude large monthly rent etc. if miscategorized)
    monthly_pattern_df = df[
        df['category'].isin(['Grocery', 'Shopping']) & (df['amount'] < 20000) # Avoid huge one-offs like Furniture if in Shopping
    ]

    if not monthly_pattern_df.empty:
        daily_total = monthly_pattern_df.groupby('day_of_month')['amount'].sum().reset_index() # Look at SUM not mean per day

        fig_month_day = px.bar(daily_total, x='day_of_month', y='amount',
                               title='Total Grocery/Shopping Spend by Day of Month',
                               labels={'day_of_month': 'Day of Month', 'amount': 'Total Amount (INR)'},
                               height=400)
        fig_month_day.update_layout(xaxis=dict(tickmode='linear', dtick=1)) # Ensure all days are shown
        fig_month_day.show() # Display inline
        # try: fig_month_day.write_image(IMG_DIR / "monthly_day_spend_pattern.png") # Commented out
        # except Exception as e: logging.error(f"Failed to save monthly pattern plot: {e}")

        # Discrepancy Check (Visual Inspection + Quantitative)
        logging.info("Check bar chart: Is spending clearly higher in the first ~10 days?")
        total_first_10 = daily_total[daily_total['day_of_month'] <= 10]['amount'].sum()
        total_last_10 = daily_total[daily_total['day_of_month'] >= 22]['amount'].sum()
        total_overall = daily_total['amount'].sum()

        if total_overall > 0 and total_first_10 > total_last_10 * 1.3: # Check if first 10 days sum is >30% higher than last 10
             logging.info(f"Spending concentration early in the month observed (First 10 days: ~₹{total_first_10:,.0f} vs Last 10 days: ~₹{total_last_10:,.0f}).")
        elif total_overall > 0:
             logging.warning(f"Grocery/Shopping spending doesn't seem strongly concentrated early in the month (First 10: ~₹{total_first_10:,.0f} vs Last 10: ~₹{total_last_10:,.0f}).")
        else:
            logging.info("Not enough spending data to assess intra-month pattern.")

    else:
        logging.info("No relevant Grocery or Shopping data found for monthly pattern analysis.")
    # Clean up added column
    if 'day_of_month' in df.columns: df.drop(columns=['day_of_month'], inplace=True)
else:
    print("Skipping intra-month pattern analysis.")


--- Intra-Month Spending Pattern (Grocery/Shopping) ---


INFO: Check bar chart: Is spending clearly higher in the first ~10 days?
INFO: Spending concentration early in the month observed (First 10 days: ~₹687,435 vs Last 10 days: ~₹0).


## 15. ML Suitability Check: Regression Potential (Flight Cost vs Month)

Explore potential relationship for regression model.

In [34]:
if not df.empty and 'date_dt' in df.columns and 'sub_category' in df.columns:
    print("\n--- ML Check: Regression Potential (Flight Cost vs Month) ---")
    flight_df = df[df['sub_category'] == 'Flight'].copy()

    if not flight_df.empty:
        flight_df['month_num'] = flight_df['date_dt'].dt.month
        fig_scatter_flight = px.scatter(flight_df, x='month_num', y='amount',
                                        title='Flight Cost vs. Month of Booking',
                                        labels={'month_num': 'Month (1-12)', 'amount': 'Flight Amount (INR)'},
                                        trendline='ols', # Ordinary Least Squares trendline
                                        height=400)
        fig_scatter_flight.update_layout(xaxis = dict(tickmode = 'array', tickvals = list(range(1,13)))) # Show all months 1-12
        fig_scatter_flight.show() # Display inline
        # try: fig_scatter_flight.write_image(IMG_DIR / "flight_cost_vs_month_scatter.png") # Commented out
        # except Exception as e: logging.error(f"Failed to save flight scatter plot: {e}")

        # Discrepancy Check (Visual Inspection)
        logging.info("Check scatter plot: Is there a visible trend (even slight)? Are outliers plausible flight costs?")
    else:
        logging.info("No flight data found for regression check.")
else:
    print("Skipping regression potential check.")


--- ML Check: Regression Potential (Flight Cost vs Month) ---


INFO: Check scatter plot: Is there a visible trend (even slight)? Are outliers plausible flight costs?


## 16. ML Suitability Check: Classification Potential (Restaurant Spend Dist.)

Look for potential thresholds to classify meals vs. snacks.

In [35]:
if not df.empty and 'category' in df.columns and 'amount' in df.columns:
    print("\n--- ML Check: Classification Potential (Restaurant Spend Distribution) ---")
    resto_df = df[df['category'] == 'Restaurant']

    if not resto_df.empty:
        fig_hist_resto = px.histogram(resto_df, x='amount', nbins=50, # More bins might show detail
                                     title='Distribution of Restaurant Transaction Amounts',
                                     labels={'amount': 'Amount (INR)'}, height=400)
        fig_hist_resto.show() # Display inline
        # try: fig_hist_resto.write_image(IMG_DIR / "restaurant_amount_histogram.png") # Commented out
        # except Exception as e: logging.error(f"Failed to save restaurant histogram: {e}")

        # Discrepancy Check (Visual Inspection)
        logging.info("Check histogram: Is there a clear separation or multiple peaks suggesting distinct groups (e.g., snacks vs meals)? Or is it one large peak skewed right?")
    else:
        logging.info("No Restaurant data found for classification check.")
else:
    print("Skipping classification potential check.")


--- ML Check: Classification Potential (Restaurant Spend Distribution) ---


INFO: Check histogram: Is there a clear separation or multiple peaks suggesting distinct groups (e.g., snacks vs meals)? Or is it one large peak skewed right?


## 17. ML Suitability Check: Segmentation Potential (Weekday vs Weekend)

Compare spending profiles across predefined day type segments.

In [36]:
if not df.empty and 'day_of_week' in df.columns and 'category' in df.columns:
    print("\n--- ML Check: Segmentation Potential (Weekday vs Weekend Profiles) ---")
    df['day_type'] = df['day_of_week'].apply(lambda x: 'Weekend' if x in ['Saturday', 'Sunday'] else 'Weekday')

    # Compare category distribution using normalized values within each segment
    segment_cat_dist = df.groupby(['day_type', 'category'])['amount'].sum().unstack(fill_value=0)
    segment_cat_dist_perc = segment_cat_dist.apply(lambda x: x / x.sum() * 100, axis=1).stack().reset_index(name='percentage')

    fig_segment_bar = px.bar(segment_cat_dist_perc, x='category', y='percentage', color='day_type',
                             barmode='group',
                             title='Category Spending %: Weekday vs. Weekend',
                             labels={'category': 'Category', 'percentage': '% of Segment Total Spend'},
                             height=500)
    fig_segment_bar.update_layout(xaxis_tickangle=-90, yaxis_title='% of Segment Spending', legend_title="Day Type")
    fig_segment_bar.show() # Display inline
    # try: fig_segment_bar.write_image(IMG_DIR / "weekday_weekend_segment_compare.png") # Commented out
    # except Exception as e: logging.error(f"Failed to save segmentation comparison plot: {e}")


    # Discrepancy Check (Visual Inspection)
    logging.info("Check grouped bar chart: Are % allocations significantly different (e.g., higher Restaurant/Entertainment % on Weekends)?")

    # Clean up added column
    if 'day_type' in df.columns: df.drop(columns=['day_type'], inplace=True)

else:
    print("Skipping segmentation potential check.")


--- ML Check: Segmentation Potential (Weekday vs Weekend Profiles) ---


INFO: Check grouped bar chart: Are % allocations significantly different (e.g., higher Restaurant/Entertainment % on Weekends)?


## 18. ML Suitability Check: Clustering Potential (Daily Spend/Activity)

Visualize daily aggregates to see if natural clusters appear.

In [37]:
if not df.empty and 'date_dt' in df.columns:
    print("\n--- ML Check: Clustering Potential (Daily Spend vs. Activity) ---")
    # Aggregate total amount and count transactions per day
    # Need a unique identifier per row if 'id' isn't reliably generated/present; using index as fallback.
    if 'id' not in df.columns: df['id_temp'] = df.index # Temporary ID if needed
    else: df['id_temp'] = df['id'] # Use existing id if present

    daily_summary = df.groupby(df['date_dt'].dt.date).agg(
        total_amount=('amount', 'sum'),
        transaction_count=('id_temp', 'nunique') # Count unique transactions per day
    ).reset_index().rename(columns={'date_dt': 'date'})


    # Clean up temp id column
    if 'id_temp' in df.columns: df.drop(columns=['id_temp'], inplace=True)

    if not daily_summary.empty:
        fig_scatter_cluster = px.scatter(daily_summary, x='transaction_count', y='total_amount',
                                        title='Daily Spending vs. Number of Transactions',
                                        labels={'transaction_count': 'Number of Transactions', 'total_amount': 'Total Amount Spent (INR)'},
                                        hover_data=['date'],
                                        height=500)
        fig_scatter_cluster.show() # Display inline
        # try: fig_scatter_cluster.write_image(IMG_DIR / "daily_spend_activity_scatter.png") # Commented out
        # except Exception as e: logging.error(f"Failed to save daily cluster scatter plot: {e}")

        # Discrepancy Check (Visual Inspection)
        logging.info("Check scatter plot: Do distinct visual groups (clusters) appear based on density? (e.g., low-count/low-spend corner, high-spend outliers, high-activity band).")
    else:
        logging.info("Could not generate daily summary for clustering check.")
else:
    print("Skipping clustering potential check.")


--- ML Check: Clustering Potential (Daily Spend vs. Activity) ---


INFO: Check scatter plot: Do distinct visual groups (clusters) appear based on density? (e.g., low-count/low-spend corner, high-spend outliers, high-activity band).


## 19. Overall Summary & Conclusion

Final assessment based on the EDA checks.

In [38]:
print("\n--- EDA Summary ---")
logging.info("Review the logs and plots above for ERRORs or WARNINGs indicating discrepancies.")
logging.info("Key areas checked: Data loading, date range/formats, nulls, row/amount constraints, User/Account mapping, Category/SubCat mapping, fixed expense accuracy, variable amount distributions, ad-hoc frequencies, account usage bias, temporal patterns, and basic visual checks for ML suitability.")
logging.info("Address critical ERRORs (e.g., mapping inconsistencies, constraint violations) by refining data generation script/rules and regenerating.")
logging.info("Consider if WARNINGs (e.g., slight constraint overshoot, weaker-than-expected patterns) are acceptable or require tuning.")
logging.info("Overall assessment: [MANUALLY ADD ASSESSMENT HERE - e.g., 'Data generated successfully and largely adheres to rules. Minor warnings noted for monthly totals. Suitable for Phase 2 development.', OR 'Critical errors found in mappings and frequencies; regeneration required.']")

INFO: Review the logs and plots above for ERRORs or WARNINGs indicating discrepancies.
INFO: Key areas checked: Data loading, date range/formats, nulls, row/amount constraints, User/Account mapping, Category/SubCat mapping, fixed expense accuracy, variable amount distributions, ad-hoc frequencies, account usage bias, temporal patterns, and basic visual checks for ML suitability.
INFO: Address critical ERRORs (e.g., mapping inconsistencies, constraint violations) by refining data generation script/rules and regenerating.
INFO: Consider if WARNINGs (e.g., slight constraint overshoot, weaker-than-expected patterns) are acceptable or require tuning.
INFO: Overall assessment: [MANUALLY ADD ASSESSMENT HERE - e.g., 'Data generated successfully and largely adheres to rules. Minor warnings noted for monthly totals. Suitable for Phase 2 development.', OR 'Critical errors found in mappings and frequencies; regeneration required.']



--- EDA Summary ---
